# **AI-Augmented Thematic Analysis (AATA) Workflow**

**Paper:** An Approach to AI-Augmented Thematic Analysis for Large-Scale Systematic Literature Reviews in Management Science  
**Author:** Wisu Suntoyo

---

### **Objective**
This notebook provides the complete code and a step-by-step guide to replicate the AI-Augmented Thematic Analysis (AATA) method. The workflow is designed to be accessible to researchers without a specialized cloud computing environment, leveraging common tools:

*   **Google Colab:** For running the Python script.
*   **Google Sheets:** For managing the source and destination datasets.
*   **Google AI's Gemini API:** For the generative AI analysis.

### **Authentication and API Keys**

To use this notebook, you will need to provide authentication credentials for both Google Colab (to access Google Sheets) and the Gemini API.

**Instructions:**

1.  **Google AI API Key:**
    *   Visit [Google AI Studio](https://aistudio.google.com/app/apikey) to create your free API key.
    *   In this Colab notebook, click the **"Key" (🔑) icon** in the left sidebar to open the "Secrets" tab.
    *   Create a new secret named `GOOGLE_API_KEY` and paste your API key into the value field. Ensure the "Notebook access" toggle is enabled.

2.  **Google Sheets Keys:**
    *   Create two Google Sheets: one for your **source data** and one for the **destination results**.
    *   For each sheet, find the "key" in its URL (the long string of characters between `/d/` and `/edit`).
    *   In the Colab "Secrets" tab, create two more secrets:
        *   `SOURCE_SPREADSHEET_KEY`: Paste the key for your source data sheet.
        *   `DESTINATION_SPREADSHEET_KEY`: Paste the key for your blank destination sheet.

3.  **Google Colab Authentication:**
    *   The code below will prompt you to authenticate your Google account. This is necessary to grant the notebook permission to read and write to your Google Sheets. Simply follow the pop-up instructions.

In [ ]:
!pip install gspread gspread-dataframe -q

import gspread
import pandas as pd
from google.colab import auth
from google.colab import userdata
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

def validate_dataframe(df, required_columns):
    """
    Validates the structure of the input DataFrame against a set of requirements.

    Args:
        df (pd.DataFrame): The DataFrame to validate.
        required_columns (list): A list of column names that must be present.

    Raises:
        ValueError: If the DataFrame is empty or if any required columns are missing.

    Returns:
        None: If validation is successful.
    """
    # 1. Check if the DataFrame is empty (no rows after the header)
    if df.empty:
        raise ValueError("Validation failed: The Google Sheet appears to be empty or has no data rows.")

    # 2. Check for missing columns
    actual_columns = set(df.columns)
    required_set = set(required_columns)
    missing_columns = required_set - actual_columns

    if missing_columns:
        # Raise an error with a specific, helpful message
        raise ValueError(f"Validation failed: The Google Sheet is missing the following required columns: {list(missing_columns)}")

    # 3. (Optional) Check if required columns are completely empty and issue a warning
    for col in required_columns:
        if df[col].isnull().all():
            print(f"Warning: The required column '{col}' exists but contains no data.")

    print("✅ DataFrame validation successful. All required columns are present.")


# Define the columns needed for analysis
REQUIRED_COLUMNS = ['Title', 'Abstract', 'Index Keywords']

try:
    # Authenticate and set up the Google Sheets client
    auth.authenticate_user()
    import google.auth
    creds, _ = google.auth.default()
    gc = gspread.authorize(creds)

    # Replace with your spreadsheet key
    SPREADSHEET_KEY = userdata.get('SPREADSHEET_KEY')

    print("Opening Google Sheet...")
    sh = gc.open_by_key(SPREADSHEET_KEY)

    # Select the first worksheet
    worksheet = sh.sheet1

    # Get all data and convert to a DataFrame
    print("Loading data into DataFrame...")
    data = worksheet.get_all_values()
    if not data:
        raise ValueError("The Google Sheet is completely empty, including the header row.")

    df = pd.DataFrame(data[1:], columns=data[0])

    # --- VALIDATION STEP ---
    # Validate the DataFrame immediately after creation
    print("Validating DataFrame structure...")
    validate_dataframe(df, REQUIRED_COLUMNS)

    # If validation is successful, proceed with your work
    print("\nDisplaying the first few rows of the validated DataFrame:")
    display(df.head())

except gspread.exceptions.SpreadsheetNotFound:
    print(f"Error: The spreadsheet with key '{SPREADSHEET_KEY}' was not found. Please check the key and your permissions.")
except ValueError as e:
    # Catch the specific validation errors and print them clearly
    print(f"Error: {e}")
except Exception as e:
    # Catch any other unexpected errors during the process
    print(f"An unexpected error occurred: {e}")

Opening Google Sheet...
Loading data into DataFrame...
Validating DataFrame structure...
✅ DataFrame validation successful. All required columns are present.

Displaying the first few rows of the validated DataFrame:


,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,DOI,Link,Abstract,Author Keywords,Index Keywords,Document Type,Publication Stage,Open Access,Source,EID
0,"Leili Mirtaheri, S.L.; Movahed, N.; Shahbazian...","Leili Mirtaheri, Seyedeh Leili (35611976100); ...",35611976100; 58566949000; 56054377000; 7004091...,Cybersecurity in the age of generative AI: A s...,2026,Future Generation Computer Systems,175,,108107,,...,10.1016/j.future.2025.108107,https://www.scopus.com/inward/record.uri?eid=2...,The article discusses the transformative impac...,Cybersecurity; Generative AI; Risk management;...,Artificial intelligence; Cybersecurity; Life c...,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-105014813374
1,"Sapkota, R.; Roumeliotis, K.I.; Karkee, M.","Sapkota, Ranjan (57765294400); Roumeliotis, Ko...",57765294400; 57485784300; 22950337200,AI Agents vs. Agentic AI: A Conceptual taxonom...,2026,Information Fusion,126,,103599,,...,10.1016/j.inffus.2025.103599,https://www.scopus.com/inward/record.uri?eid=2...,"Information fusion, in the context of the Gene...",Agentic AI; AI agents; Conceptual taxonomy; Co...,Automation; Autonomous agents; Decision suppor...,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-105013867917
2,"Eccles, B.J.; Wong, L.; Varghese, B.","Eccles, Bailey J. (57987427800); Wong, Leon (5...",57987427800; 57245866300; 26435611000,Mosaic: Composite projection pruning for resou...,2026,Future Generation Computer Systems,175,,108056,,...,10.1016/j.future.2025.108056,https://www.scopus.com/inward/record.uri?eid=2...,Extensive compute and memory requirements limi...,Composite projection pruning; Edge computing; ...,Compaction; Computer hardware; Composite proje...,Article,Final,All Open Access; Green Accepted Open Access; G...,Scopus,2-s2.0-105013552766
3,"Barbosa, R.Z.; Oliveira, H.S.; Tavares, J.M.R.S.","Barbosa, Rui Z. (59737024800); Oliveira, Hugo ...",59737024800; 57214550108; 36538110300,A survey on multi-modal and weakly supervised ...,2026,Information Fusion,126,,103388,,...,10.1016/j.inffus.2025.103388,https://www.scopus.com/inward/record.uri?eid=2...,This survey provides a comprehensive overview ...,Abnormality criteria; Benchmarks; Cross-modal ...,Anomaly detection; Data integration; Data priv...,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-105013502307
4,"Napoli, E.A.; Romani, N.; Gatteschi, V.; Schif...","Napoli, Emanuele Antonio (57220991562); Romani...",57220991562; 59390420700; 37161268600; 5599075...,Light and shadows of smart contract developmen...,2026,Expert Systems with Applications,296,,129011,,...,10.1016/j.eswa.2025.129011,https://www.scopus.com/inward/record.uri?eid=2...,Smart contract development remains almost inac...,Blockchain; Large language model; Legal contra...,Laws and legislation; Network security; Smart ...,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-105011033975


Core AI analysis, researcher to make modifications to prompt to align with research

In [ ]:
import re
import json
import google.generativeai as genai
import time
import os

from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Create the generative model instance
llm = genai.GenerativeModel("gemini-flash-lite-latest")

def extract_json_from_string(text):
    """
    Finds and extracts a JSON object from a string that might be wrapped
    in markdown or other text.
    """
    match = re.search(r'\{.*\}', text, re.DOTALL)

    if match:
        json_str = match.group(0)
        return json.loads(json_str)
    else:
        # If no JSON object is found, we can't proceed.
        raise ValueError("No valid JSON object found in the model's response.")

def analyze(title, abstract, keywords):

    prompt = f"""
# CONTEXT
You are a highly-skilled research assistant for a PhD student. Your primary goal is to systematically screen academic papers
against a set of predefined inclusion criteria to determine their relevance for a dissertation. The dissertation focuses on
three key areas:
    (A) The use of Conversational AI for survey research,
    (B) Participative Decision-Making (PDM) and
    (C) Human-AI Interaction (H-AI).

# ANALYTICAL FRAMEWORK
You must evaluate each paper against the following framework:

## A. Core Research Questions (Conversational AI in Surveys)
*   **RQ1:** Is it feasible to use generative AI conversational interactions to administer research questionnaires?
*   **RQ2:** How do response rates and data quality compare between questionnaires administered using generative
             AI conversational interactions and traditional online questionnaires?
*   **RQ3:** What is the capability of scaling the population of research data collection with generative AI conversation?

## B. Core Topic (Participative Decision-Making - PDM) - CONCEPTUAL DEFINITION
* **Core Definition:** PDM is a management approach where leaders involve employees in decisions that affect their work
                       and the organization. It is a collaborative method for sharing ideas, knowledge, and perspectives
                      to improve decision quality and implementation.
* **Synonyms & Related Concepts:** Look for discussions of 'shared decision-making', 'employee involvement',
                      'stakeholder engagement', 'employee voice', 'collaborative decision-making', or 'joint decision-making'.
* **Key Indicators of PDM:** The paper may discuss these ideas without using the exact terms. Identify PDM if the paper
                      analyzes topics like:
    - Soliciting employee/user input on work priorities, new strategies, or organizational changes.
    - The benefits of collaboration on decision quality, employee motivation, job satisfaction, or organizational commitment.
    - Varying degrees of participation, from simple consultation to full empowerment in decisions.

## C. Key Term Definition (Crucial for 'CAI_focus' assessment)
*   **IMPORTANT:** For this analysis, "Conversational AI" (CAI) is a broad concept. Many papers may not use this exact term.
                   You must identify a 'present' focus on CAI if the paper explicitly discusses or studies technologies
                   such as **chatbots, conversational agents, generative AI, large language models (LLMs),
                   or virtual assistants.** This includes, but is not limited to, specific tools like
                   **ChatGPT, Gemini, Claude, etc.** The key is whether the technology facilitates a multi-turn,
                   natural language dialogue with a user. A paper that only mentions 'AI' in a generic sense for
                   data analysis without a dialogue component does NOT qualify.

## D. Additional Inclusion Criteria (Definitions for Classification)
* **Web_survey_criticism:** Assess the extent to which the paper critiques traditional web surveys.
  - **strong:** The paper's core argument or justification relies on the stated flaws of web surveys
                (e.g., low engagement, satisficing behavior, poor data quality).
  - **moderate:** The paper discusses the limitations of web surveys as important context, typically in the introduction
                or literature review.
  - **limited:** A brief, passing mention of web survey limitations.
  - **none:** No criticism of web surveys is mentioned.

* **User_perspectives:** Assess the extent to which the paper's methodology and findings are focused on the end-user's
                         experience, feelings, or opinions.
  - **strong:** The primary data comes from users (e.g., qualitative interviews, UX studies, satisfaction surveys,
                think-aloud protocols). The findings are about what users think or feel.
  - **moderate:** User perspectives are a component of the study, but are combined with other data types (e.g.,
                  technical performance metrics).
  - **limited:** The paper discusses implications for users but does not collect direct data from them.
  - **none:** The paper is purely technical, theoretical, or architectural, with no focus on the end-user's
              subjective experience.

* **Human_AI_Interaction_focus:** Assess if the paper analyzes the *process* of interaction between the human
                                  and the AI. This is distinct from just measuring user satisfaction *after* the fact.
  - **strong:** The paper explicitly analyzes the dynamics of the dialogue itself (e.g., conversational turns,
                sensemaking processes, trust formation during interaction, communication patterns, prompt
                engineering analysis from a user perspective). The interaction is a central object of study.
  - **moderate:** The paper describes the conversational flow or interaction design but does not deeply analyze it.
                  It may measure outcomes (like task success) that result from the interaction without analyzing the
                  interaction process itself.
  - **limited:** A brief, passing mention that users "interact" or "chat" with the system.
  - **none:** The study does not involve or discuss a back-and-forth dialogue between a human and an AI.

* **PDM_discussion:** Based on the conceptual definition in section B, assess the extent to which the paper
                      discusses PDM. The context is critical.
  - **strong:** The paper discusses PDM or its related concepts (e.g., 'shared decision-making', 'employee voice')
                as a central theme, motivation, or key implication ithin an organizational context
                (e.g., corporate, enterprise or workplace management).
  - **moderate:** The paper discusses PDM concepts, but either **outside of an organizational management context**
                (e.g., public sector or a clinical patient-doctor setting) or as a secondary point within an
                organizational context.
  - **limited:** A brief, passing mention of 'decision-making' in a general sense without a clear participatory element.
  - **none:** PDM or related concepts are not discussed.

* **Organizational_context:** Determine if the paper's discussion of any topic situated within a corporate, workplace,
                              or employee-management setting.
  - **true:** The context is clearly organizational (e.g., a company, a public sector agency, employee teams,
              management strategies).
  - **false:** The context is non-organizational (e.g., clinical patient-doctor interactions, general public
              participation with government, generic design workshops).

# INPUT MATERIALS
You will analyze the following materials for a single research paper:
*   **Title:** {title}
*   **Keywords:** {keywords}
*   **Abstract:** {abstract}

# TASK
Follow these steps precisely, using all definitions provided in the Analytical Framework. For each paper,
assess its similarity to the RQs and classify it against each of the inclusion criteria.

# OUTPUT INSTRUCTIONS
You MUST provide your response as a single, valid JSON object literal without any explanatory text before or after it.

Your JSON response must conform exactly to the following structure and use ONLY the specified enumerated values for each key.

{{
  "RQ1_Similarity": "enum [strong, potential, limited, none]",
  "RQ2_Similarity": "enum [strong, potential, limited, none]",
  "RQ3_Similarity": "enum [strong, potential, limited, none]",
  "CAI_focus": "enum [true, false]",
  "Initial_RQ_relevance": "enum [strong, moderate, limited, none]",
  "Web_survey_criticism": "enum [strong, moderate, limited, none]",
  "User_perspectives": "enum [strong, moderate, limited, none]",
  "Human_AI_Interaction_focus": "enum [strong, moderate, limited, none]",
  "PDM_discussion": "enum [strong, moderate, limited, none]"
  "Organizational_context": "enum [true, false]",
}}
"""

    generation_config = genai.types.GenerationConfig(
        max_output_tokens=8192,
        temperature=0,
        top_p=1,
        response_mime_type="application/json"
    )

    safety_settings = {
        "HARM_CATEGORY_HATE_SPEECH": "BLOCK_NONE",
        "HARM_CATEGORY_DANGEROUS_CONTENT": "BLOCK_NONE",
        "HARM_CATEGORY_SEXUALLY_EXPLICIT": "BLOCK_NONE",
        "HARM_CATEGORY_HARASSMENT": "BLOCK_NONE",
    }

    try:
        response = llm.generate_content(
            contents=[prompt],
            generation_config=generation_config,
            safety_settings=safety_settings,
        )

        print(response.text)

        return extract_json_from_string(response.text)

    except (json.JSONDecodeError, ValueError) as e:
        print(f"Error parsing JSON from model output: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during analysis: {e}")
        return None

def analyze_and_add_columns(paper):
    """
    Dynamically analyzes a paper and adds the results as new columns.

    This function calls the 'analyze' function to get a JSON/dict of analysis results.
    It then iterates through the key-value pairs in the result and adds them
    as new columns to the input 'paper' Series. This makes the function
    automatically adapt to any changes in the JSON structure defined in the prompt.
    """
    start = time.time()
    try:
        # 1. Call the analyze function as before to get the analysis dictionary
        paper_analysis = analyze(
            abstract=paper["Abstract"],
            title=paper["Title"],
            keywords=paper["Index Keywords"]
        )

        # 2. Dynamically iterate through the returned dictionary
        #    This replaces all the hard-coded lines.
        for key, value in paper_analysis.items():
            paper[key] = value

        print(f"Successfully analyzed: {paper['Title']}")

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for '{paper['Title']}': {e}")
    except Exception as e:
        print(f"An unexpected error occurred for '{paper['Title']}': {e}")

    done = time.time()
    elapsed = done - start
    print(f"Time elapsed: {elapsed:.2f} seconds")

    return paper

### **Executing the Main Analysis Workflow**

This is the main execution block of the notebook. It has been designed to be robust, allowing you to process a large corpus of papers without losing work due to common disruptions.

#### **Key Features of this Workflow:**

*   **Automatic Resumption:** The most critical feature is the ability to automatically resume progress. Before starting a new run, the script connects to your destination Google Sheet and checks for existing data. It will automatically skip any rows that have already been processed and begin the analysis from the first empty row. **This ensures that no progress is lost, even after an interruption.**

*   **Batch Processing:** The analysis is performed in batches (e.g., 100 papers at a time). Once a batch is successfully analyzed, the results are immediately appended as new rows to the destination Google Sheet, saving your progress incrementally.

*   **Data Integrity:** Data is loaded fresh from the source Google Sheet at the start of each run, and a destination sheet must be available and configured in the setup cells.

#### **Handling Interruptions (e.g., API Timeouts, Idle Screens)**

Long-running browser sessions are often disrupted by API timeouts, Colab idle screen disconnections, or other network issues. This script is designed to handle such interruptions gracefully.

**If the process stops unexpectedly, follow this simple recovery procedure:**

1.  From the Colab menu, select `Runtime` -> `Disconnect and delete runtime`. This ensures a clean start.
2.  Rerun the cells in Sections 1 and 2 to re-authenticate and reload the data.
3.  Rerun this main execution cell. The script will automatically detect your previous progress in the destination sheet and conveniently resume from exactly where it left off.

**Performance Benchmark:** With the `gemini-flash-lite-latest` model, the typical analysis time is approximately **1.05 seconds per paper**. Including API call overhead and pauses between batches, a complete run on a corpus of **1,000 papers** is estimated to take **approximately 17 minutes**.

In [ ]:
import time
from gspread_dataframe import get_as_dataframe
import gspread.exceptions

# --- CONFIGURATION ---
BATCH_SIZE = 100
DESTINATION_SPREADSHEET_KEY = userdata.get('DESTINATION_SPREADSHEET_KEY')

# --- CHANGE #1: DYNAMICALLY DETERMINE ANALYSIS COLUMNS ---
print("--- Probing the analysis function to determine output columns ---")
analysis_columns = []
try:
    # Perform a single test call with dummy data to get the expected keys
    sample_analysis = analyze(title="test title", abstract="test abstract", keywords="test keywords")

    if sample_analysis and isinstance(sample_analysis, dict):
        # The keys from the returned dictionary are our new columns
        analysis_columns = list(sample_analysis.keys())
        print(f"✅ Successfully determined analysis columns: {analysis_columns}")
    else:
        # If the API returns None or not a dictionary, we can't proceed
        raise ValueError("Analysis probe did not return a valid dictionary. Check the 'analyze' function or API response.")

except Exception as e:
    print(f"🚨 CRITICAL ERROR: Could not determine analysis columns automatically. The script cannot proceed.")
    print(f"   Error details: {e}")
    # Re-raise the exception to stop the script execution
    raise

# First, get the original columns from your source DataFrame
original_columns = df.columns.tolist()

# Combine them to create the final, ordered list of all columns
FINAL_COLUMNS = original_columns + analysis_columns
print(f"Final column order has been set with {len(FINAL_COLUMNS)} columns.")


# --- SETUP DESTINATION SHEET ---
print("\n--- Setting up destination sheet ---")
try:
    sh_to_save = gc.open_by_key(DESTINATION_SPREADSHEET_KEY)
    worksheet_to_save = sh_to_save.sheet1
    print(f"Successfully opened destination Google Sheet: '{sh_to_save.title}'")
except gspread.exceptions.SpreadsheetNotFound as e:
    print(f"ERROR: Destination spreadsheet with key '{DESTINATION_SPREADSHEET_KEY}' not found.")
    raise

# --- DETERMINE STARTING POINT ---
try:
    existing_data = worksheet_to_save.get_all_values()
    # The start_index should be the number of DATA rows, which is total rows - 1 (for the header)
    if len(existing_data) > 1:
        start_index = len(existing_data) - 1
        print(f"Found {start_index} existing data rows. Resuming...")
    else:
        start_index = 0
        print("Destination sheet is empty or has only a header. Starting from scratch.")
except Exception:
    start_index = 0
    print("Could not read from destination sheet (it may be new). Starting from scratch.")


# --- MAIN BATCH PROCESSING LOOP ---
total_rows = len(df)
print(f"\n--- Starting analysis of {total_rows - start_index} remaining rows ---")

for i in range(start_index, total_rows, BATCH_SIZE):
    end_index = min(i + BATCH_SIZE, total_rows)
    print(f"\nProcessing batch: rows {i} to {end_index-1}...")

    batch = df.iloc[i:end_index]
    analyzed_batch = batch.apply(analyze_and_add_columns, axis=1)

    # --- HEADER MANAGEMENT ---
    if i == 0 and start_index == 0:
        worksheet_to_save.clear()
        worksheet_to_save.update('A1', [FINAL_COLUMNS])
        print("Wrote header to destination sheet using predefined column order.")

    # --- RE-ORDER BATCH and APPEND ---
    consistent_batch = analyzed_batch.reindex(columns=FINAL_COLUMNS)
    rows_to_append = consistent_batch.fillna('').values.tolist()

    if rows_to_append:
        try:
            worksheet_to_save.append_rows(rows_to_append, value_input_option='USER_ENTERED')
            print(f"✅ Successfully saved batch (rows {i}-{end_index-1}) to Google Sheet.")
        except gspread.exceptions.APIError as e:
            print(f"🚨 ERROR saving batch to Google Sheets: {e}")
            print("Stopping execution. You can restart the script to resume from this point.")
            break

    # --- RATE LIMITING ---
    print("Pausing for 5 seconds to respect API rate limits...")
    time.sleep(5)

print("\n--- All batches processed. ---")

Streaming output truncated to the last 5000 lines.
{
  "RQ1_Similarity": "none",
  "RQ2_Similarity": "none",
  "RQ3_Similarity": "none",
  "CAI_focus": "false",
  "Initial_RQ_relevance": "none",
  "Web_survey_criticism": "limited",
  "User_perspectives": "limited",
  "Human_AI_Interaction_focus": "none",
  "PDM_discussion": "moderate",
  "Organizational_context": "false"
}
Successfully analyzed: How to approach narratives in foresight studies: Qualitative data analysis
Time elapsed: 1.23 seconds
{
  "RQ1_Similarity": "none",
  "RQ2_Similarity": "none",
  "RQ3_Similarity": "none",
  "CAI_focus": "false",
  "Initial_RQ_relevance": "none",
  "Web_survey_criticism": "none",
  "User_perspectives": "limited",
  "Human_AI_Interaction_focus": "none",
  "PDM_discussion": "moderate",
  "Organizational_context": "false"
}
Successfully analyzed: Diversity and constructive conflict in stakeholder dialogue: Considerations for design and methods
Time elapsed: 1.26 seconds
{
  "RQ1_Similarity": "none"

### **Statistics of Analysis**

In [ ]:
# This cell installs the required libraries for accessing Google Sheets and creating plots.
!pip install gspread google-auth-oauthlib pandas plotly -q

import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
from google.colab import userdata

# This cell handles authentication with your Google account to access the spreadsheet.
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# --- YOUR INPUT REQUIRED ---
# Please replace 'YOUR_GOOGLE_SHEET_KEY_HERE' with the key from your Google Sheet URL
DESTINATION_SPREADSHEET_KEY = userdata.get('DESTINATION_SPREADSHEET_KEY')
# -------------------------

# Open the spreadsheet and load the data into a pandas DataFrame
try:
    worksheet = gc.open_by_key(DESTINATION_SPREADSHEET_KEY).sheet1
    rows = worksheet.get_all_values()
    df = pd.DataFrame.from_records(rows)
    # Set the first row as the header
    df.columns = df.iloc[0]
    df = df.iloc[1:]
    print(f"Successfully loaded {len(df)} records from your Google Sheet.")
    print("Columns found:", list(df.columns))
except Exception as e:
    print(f"An error occurred: {e}")
    print("Please ensure you have entered a valid Google Sheet key and have shared the sheet with your authenticated user account.")

Successfully loaded 10358 records from your Google Sheet.
Columns found: ['Authors', 'Author full names', 'Author(s) ID', 'Title', 'Year', 'Source title', 'Volume', 'Issue', 'Art. No.', 'Page start', 'Page end', 'Page count', 'Cited by', 'DOI', 'Link', 'Abstract', 'Author Keywords', 'Index Keywords', 'Document Type', 'Publication Stage', 'Open Access', 'Source', 'EID', 'RQ1_Similarity', 'RQ2_Similarity', 'RQ3_Similarity', 'CAI_focus', 'Initial_RQ_relevance', 'Web_survey_criticism', 'User_perspectives', 'Human_AI_Interaction_focus', 'PDM_discussion', 'Organizational_context']


In [ ]:
# This function calculates the counts for our funnel stages
def analyze_funnel(dataframe):
    """
    Analyzes the dataframe to create a funnel based on predefined criteria.
    The funnel moves from broad to specific inclusion criteria.
    Also captures and returns excluded papers at specific stages.
    """
    print("\n--- Starting Funnel Analysis ---\n")

    funnel_data = []
    # Start with the full dataframe passed to the function
    current_df = dataframe.copy()
    print(f"Initial records for analysis: {len(current_df)}\n")

    # --- Stage 1: CAI_focus ---
    total_at_stage = len(current_df)
    strong_matches = current_df[current_df['CAI_focus'] == 'TRUE']
    num_strong = len(strong_matches)
    funnel_data.append({'Stage': '1. CAI Focus', 'Count': num_strong})
    print(f"Stage 1: CAI_focus = TRUE")
    print(f"  - Matches: {num_strong} papers")
    print(f"  - Remaining from {total_at_stage} papers\n")
    current_df = strong_matches

    # --- Stage 2: Organizational_context ---
    total_at_stage = len(current_df)
    strong_matches = current_df[current_df['Organizational_context'] == 'TRUE']
    num_strong = len(strong_matches)
    funnel_data.append({'Stage': '2. Organizational Context', 'Count': num_strong})
    print(f"Stage 2: Organizational_context = TRUE")
    print(f"  - Matches: {num_strong} papers")
    print(f"  - Remaining from {total_at_stage} papers\n")
    current_df = strong_matches

    # --- Stage 3: User_perspectives ---
    total_at_stage = len(current_df)
    strong_matches = current_df[current_df['User_perspectives'] == 'strong']
    num_strong = len(strong_matches)
    # Calculate others for reporting
    others = {
        'moderate': len(current_df[current_df['User_perspectives'] == 'moderate']),
        'limited': len(current_df[current_df['User_perspectives'] == 'limited']),
        'none': len(current_df[current_df['User_perspectives'] == 'none'])
    }
    funnel_data.append({'Stage': '3. User Perspectives (Strong)', 'Count': num_strong})
    print(f"Stage 3: User_perspectives = 'strong'")
    print(f"  - 'Strong' Matches: {num_strong} papers")
    print(f"  - Other categories: Moderate({others['moderate']}), Limited({others['limited']}), None({others['none']})")
    print(f"  - Remaining from {total_at_stage} papers\n")
    current_df = strong_matches

    # --- Stage 4: Human_AI_Interaction_focus ---
    total_at_stage = len(current_df)
    strong_matches_stage4 = current_df[current_df['Human_AI_Interaction_focus'] == 'strong']
    num_strong_stage4 = len(strong_matches_stage4)
    others_stage4 = {
        'moderate': len(current_df[current_df['Human_AI_Interaction_focus'] == 'moderate']),
        'limited': len(current_df[current_df['Human_AI_Interaction_focus'] == 'limited']),
        'none': len(current_df[current_df['Human_AI_Interaction_focus'] == 'none'])
    }
    funnel_data.append({'Stage': '4. H-AI Interaction Focus (Strong)', 'Count': num_strong_stage4})
    print(f"Stage 4: Human_AI_Interaction_focus = 'strong'")
    print(f"  - 'Strong' Matches: {num_strong_stage4} papers")
    print(f"  - Other categories: Moderate({others_stage4['moderate']}), Limited({others_stage4['limited']}), None({others_stage4['none']})")
    print(f"  - Remaining from {total_at_stage} papers\n")

    # Capture excluded papers at Stage 4 and display Title, Authors, and DOI
    # excluded_stage4 = current_df[current_df['Human_AI_Interaction_focus'] != 'strong']
    # print("Papers excluded at Stage 4 (H-AI Interaction Focus != 'strong'):")
    # if not excluded_stage4.empty:
    #     # Display selected columns for excluded papers
    #     display_cols = ['Title']
    #     # if 'Authors' in excluded_stage4.columns:
    #     #     display_cols.append('Authors')
    #     if 'DOI' in excluded_stage4.columns:
    #         display_cols.append('DOI')

    #     if display_cols:
    #         print(excluded_stage4[display_cols].to_string(index=False))
    #     else:
    #         print("  (No relevant columns to display)")
    # else:
    #     print("No papers excluded at this stage.")
    # print("-" * 50)

    current_df = strong_matches_stage4

    # --- Stage 5: PDM_discussion ---
    total_at_stage = len(current_df)
    strong_matches_stage5 = current_df[current_df['PDM_discussion'] == 'strong']
    num_strong_stage5 = len(strong_matches_stage5)
    others_stage5 = {
        'moderate': len(current_df[current_df['PDM_discussion'] == 'moderate']),
        'limited': len(current_df[current_df['PDM_discussion'] == 'limited']),
        'none': len(current_df[current_df['PDM_discussion'] == 'none'])
    }
    funnel_data.append({'Stage': '5. PDM Discussion (Strong)', 'Count': num_strong_stage5})
    print(f"\nStage 5: PDM_discussion = 'strong'")
    print(f"  - 'Strong' Matches: {num_strong_stage5} papers")
    print(f"  - Other categories: Moderate({others_stage5['moderate']}), Limited({others_stage5['limited']}), None({others_stage5['none']})")
    print(f"  - This is the final, most relevant set of papers from the initial {len(dataframe)}.\n")

    # Capture excluded papers at Stage 5 and display Title, Authors, and DOI
    excluded_stage5 = current_df[current_df['PDM_discussion'] != 'strong']
    print("Papers excluded at Stage 5 (PDM Discussion != 'strong'):")
    if not excluded_stage5.empty:
         # Display selected columns for excluded papers
        display_cols = ['Title']
        if 'DOI' in excluded_stage5.columns:
            display_cols.append('DOI')
        if 'Authors' in excluded_stage5.columns:
            display_cols.append('Authors')

        if display_cols:
            print(excluded_stage5[display_cols].to_string(index=False))
        else:
            print("  (No relevant columns to display)")
    else:
        print("No papers excluded at this stage.")

    return funnel_data

# Run the analysis only if the dataframe was loaded successfully
if 'df' in locals() and not df.empty:
    funnel_results = analyze_funnel(df)
else:
    print("DataFrame not loaded. Please check Step 2 for errors.")


--- Starting Funnel Analysis ---

Initial records for analysis: 10358

Stage 1: CAI_focus = TRUE
  - Matches: 5273 papers
  - Remaining from 10358 papers

Stage 2: Organizational_context = TRUE
  - Matches: 379 papers
  - Remaining from 5273 papers

Stage 3: User_perspectives = 'strong'
  - 'Strong' Matches: 63 papers
  - Other categories: Moderate(148), Limited(126), None(42)
  - Remaining from 379 papers

Stage 4: Human_AI_Interaction_focus = 'strong'
  - 'Strong' Matches: 11 papers
  - Other categories: Moderate(23), Limited(28), None(1)
  - Remaining from 63 papers


Stage 5: PDM_discussion = 'strong'
  - 'Strong' Matches: 0 papers
  - Other categories: Moderate(0), Limited(0), None(11)
  - This is the final, most relevant set of papers from the initial 10358.

Papers excluded at Stage 5 (PDM Discussion != 'strong'):
                                                                                                                                                                      